# College Admission

## Data Preprocessing
Load libraries  
Load csv file to dataframe

In [ ]:
import numpy as np
import pandas as pd
df = pd.read_csv('Admission_Predict_Ver1.1.csv', index_col=0)

Convert quantitative data ('Chance of Admit') into categorical ('Chance')

In [8]:
bins = [0, 0.2, 0.4, 0.6, 0.8, 1]
names = ['very low', 'moderately low', 'medium', 'moderately high', 'very high']
df['Chance'] = pd.cut(df['Chance of Admit '], bins, labels=names)
df.head()

,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research,Chance of Admit,Chance
Serial No.,,,,,,,,,
1,337,118,4,4.5,4.5,9.65,1,0.92,very high
2,324,107,4,4.0,4.5,8.87,1,0.76,moderately high
3,316,104,3,3.0,3.5,8.00,1,0.72,moderately high
4,322,110,3,3.5,2.5,8.67,1,0.80,moderately high
5,314,103,2,2.0,3.0,8.21,0,0.65,moderately high
